# Manual control harness for python module
## 2018-09-23
Uses a python class to wrap the robot controller which is imported.

### To align (Servo version)

- Switch on if not on
- Run the code if not already running
- Use the command `td` to test the DVD.  This will position the tool head just above the DVD drive.
- Put a dummy CD in
- Turn the power off
- Rotate the arm by hand to be exactly over the DVD
- Turn the power on
- Restart and clear output on code (the COM port will have gone and should now be back)

## DVD door commands

In [ ]:
import string
from ctypes import windll

class DVDError(Exception):
    pass

def get_drives():
    drives = []
    bitmask = windll.kernel32.GetLogicalDrives()
    print(bitmask)
    for letter in 'ABCDEFGHIJKLMNOPQ':
        if bitmask & 1:
            drives.append(letter)
        bitmask >>= 1

    return drives

def drive_is_d():
    drive_list = get_drives()
    assert len(drive_list) == 2
    assert drive_list[1] == 'D'
    return True

def open_dvd():
    windll.WINMM.mciSendStringW(u"set cdaudio door open", None, 0, None)

def close_dvd():
    windll.WINMM.mciSendStringW(u"set cdaudio door closed", None, 0, None)

if drive_is_d():
    print('Found D drive all ok')
else:
    raise DVDError("Didn't find D drive")

54525964
Found D drive all ok


## Commands to control robot

In [ ]:
import time

import od_control

robot = od_control.ODRobot(open_dvd, close_dvd, com_port="COM6")


while 1:
    # get keyboard input
    user_input = input("->")
    if user_input == 'e':
        print('Shutting serial down')
        robot.close()
        print('Breaking loop to stop')
        break
    elif user_input == 'l':  # Load disk into drive
        robot.load()
    elif user_input == 'u':  # Unload disk from drive
        robot.unload()
    elif user_input == 'td':  # Test DVD alignment
        robot.make_safe()
        robot.disk_open()
        robot.rotate.dvd()
        robot.robot('control1.z_axis.nudge(49, run_time=30)')  # Move nearer dvd
    elif user_input == 'td1':  # Test Pickup DVD from drive and drop it back
        robot.make_safe()
        robot.pickup_from_dvd_drive()
        robot.drop_on_dvd_drive()
        robot.make_safe()
        robot.disk_close()
    elif user_input == 'tl':  # Load disk into drive
        robot.make_safe()
        robot.rotate.in_bin()
        robot.robot('control1.z_axis.nudge(49, run_time=30)')  # Move nearer dvd
    elif user_input == 'tu':  # Unload disk from drive
        robot.make_safe()
        robot.rotate.out_bin()
        robot.robot('control1.z_axis.nudge(49, run_time=30)')  # Move nearer dvd
    elif user_input == 'n':
        robot.robot('control1.th.park()')
    elif user_input == 'm':
        robot.robot('control1.th.grip()')
    elif user_input == ',':
        robot.robot('control1.th.release()')
    elif user_input == '.':
        robot.robot('control1.th.shutdown()')
    elif len(user_input) > 2 and user_input[0] == 'r':
        position = float(user_input[1:])
        print(f'New position = {position}')
        robot.robot(f'control1.rotate.wait_set_position({position})')
    elif user_input == 's':
        robot.robot('control1.status()')
    elif user_input == 's10':
        for i in range(10):
            robot.robot('control1.status()')
    else:
        robot.robot(user_input)


Robot>import control1
>>> 
->l
Robot>control1.th.park()
>>> 
Robot>control1.th.shutdown()
>>> 
Robot>control1.z_axis.nudge(44, run_time=90)
Shutdown Z Axis nudge
>>> 
Robot>control1.z_axis.shutdown()
>>> 
Robot>control1.rotate.wait_set_position(81)
>>> 
Robot>control1.rotate.wait_set_position(86)
>>> 
Robot>control1.th.release()
>>> 
Robot>control1.z_axis.nudge(49, run_time=120)

Robot>Shutdown Z Axis nudge
>>> control1.th.grip()
>>> control1.z_axis.nudge(44, run_time=90)

Robot>Shutdown Z Axis nudge
>>> control1.z_axis.shutdown()
>>> control1.rotate.wait_set_position(44)
>>> control1.rotate.wait_set_position(49)
>>> control1.z_axis.nudge(49, run_time=40)
Shutdown Z Axis nudge
>>> control1.th.release()
>>> 
Robot>control1.th.park()
>>> 
Robot>control1.th.shutdown()
>>> 
Robot>control1.th.park()
>>> 
Robot>control1.th.shutdown()
>>> 
Robot>control1.z_axis.nudge(44, run_time=90)

Robot>Shutdown Z Axis nudge
>>> control1.z_axis.shutdown()
>>> 
